In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Importing libraries

In [2]:
import os
import glob 
import keras
import shutil
import random 
import imageio 
import skimage 
import tensorflow 
import numpy as np
from PIL import Image
from skimage import io 
from skimage import data
from numpy import expand_dims
import matplotlib.pyplot as plt  
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator
%matplotlib inline

# Preprocessing and Augmentation

In [ ]:
counter = 0

In [15]:
# Importing and Loading the data into a data frame

dataset_path = '/content/drive/MyDrive/Farsi-Digit-Detection/custom-datasets/DS2/valid/images'

# apply glob module to retrieve files/pathnames  

digit_path = os.path.join(dataset_path, '*')
digit_path = glob.glob(digit_path)

In [16]:
len(digit_path) #30+240+29

29

## Resizing

In [ ]:
def resize(image):

  global counter

  #Read image
  image = Image.open(image)

  #Resize image
  image = image.resize((416,416),Image.ANTIALIAS)
  path = "/content/drive/MyDrive/Farsi-Digit-Detection/images/" + str(counter) + ".jpeg"
  counter = counter + 1

  #Save image
  image.save(path)

## Grayscale conversion

In [ ]:
def gary_scale(image):
  gray_image = skimage.color.rgb2gray(image)

  return gray_image

## Normalization

In [ ]:
def normalization(image):
  norm_image = (image - np.min(image)) / (np.max(image) - np.min(image))
  
  return norm_image

## Flipping

In [ ]:
def flipping(image):

  global counter

  # convert to numpy array
  data = img_to_array(image)

  # expand dimension to one sample
  samples = expand_dims(image, 0)

  # ImageDataGenerator for flipping
  datagen = ImageDataGenerator(horizontal_flip=True, vertical_flip=True)

  # create an iterator
  it = datagen.flow(samples, batch_size=1)

  # generate batch of images
  for i in range(3):

    # convert to unsigned integers
    image = next(it)[0].astype('uint8')

    #save generated images
    filename = "/content/drive/MyDrive/Farsi-Digit-Detection/custom-dataset/flipped-images/" + str(counter) + ".jpeg"
    counter = counter + 1
    imageio.imwrite(filename, image)
      
       

## Rotation

In [ ]:
def rotation(image):

  global counter

  # convert to numpy array
  data = img_to_array(image)

  # expand dimension to one sample
  samples = expand_dims(image, 0)

  # create an iterator
  datagen = ImageDataGenerator(rotation_range=20, fill_mode='nearest')

  # create an iterator
  it = datagen.flow(samples, batch_size=1)

  # generate batch of images
  for i in range(3):

    # convert to unsigned integers
    image = next(it)[0].astype('uint8')

    #save generated images
    filename = "/content/drive/MyDrive/Farsi-Digit-Detection/custom-dataset/pelak-dataset/rotated-images/" + str(counter) + ".jpeg"
    counter = counter + 1
    imageio.imwrite(filename, image)

## Calling Resize Function

In [ ]:
for i in range(0,len(digit_path)):
  #image = io.imread(digit_path[i])
  resize(digit_path[i])

## Calling Augmentation Functions

In [ ]:
for i in range(0,len(digit_path)): 
  image = io.imread(digit_path[i])
  try:
    #flipping(image)
    rotation(image)
  except:
    print(digit_path[i])

print("END")

END


## Calling Preprocessing Functions

In [ ]:
for i in range(0,len(digit_path)): 
  image = io.imread(digit_path[i])
  gray_image = gary_scale(image)
  norm_image = normalization(image)
  path = "/content/drive/MyDrive/Farsi-Digit-Detection/normalized-images/" + str(i) + ".jpeg"
  imageio.imwrite(path, norm_image)
print("END")

# Spiliting Dataset

In [ ]:
current_dir = "/content/drive/MyDrive/Farsi-Digit-Detection/Hoda-Dataset/images"
# Percentage of images to be used for the valid set
percentage_test = 20

flag = True

# Create train.txt and valid.txt
file_train = "/content/drive/MyDrive/Farsi-Digit-Detection/Hoda-Dataset/train" 
file_test = "/content/drive/MyDrive/Farsi-Digit-Detection/Hoda-Dataset/test"
file_valid = "/content/drive/MyDrive/Farsi-Digit-Detection/Hoda-Dataset/valid"
# Populate train.txt and valid.txt
counter = 1  
index_test = round(100 / percentage_test)  
for file in glob.iglob(os.path.join(current_dir, '*.png')):  
    title, ext = os.path.splitext(os.path.basename(file))
    cur_img = current_dir + "/" + title + '.png'
    cur_lbl = "/content/drive/MyDrive/Farsi-Digit-Detection/Hoda-Dataset/labels/"  + title + '.txt'

    if counter == index_test:
      if flag == True :
        test_img = file_test + '/images'  
        test_lbl = file_test + '/labels'  
        shutil.move(cur_img,test_img)
        shutil.move(cur_lbl,test_lbl)
      else:
        valid_img = file_valid + '/images'  
        valid_lbl = file_valid + '/labels'  
        shutil.move(cur_img,valid_img)
        shutil.move(cur_lbl,valid_lbl)
      flag = not flag
      counter = 1
    else:
        train_img = file_train + '/images'  
        train_lbl = file_train + '/labels'  
        shutil.move(cur_img,train_img)
        shutil.move(cur_lbl,train_lbl)
        counter = counter + 1